In [1]:
import torch

In [2]:
import pandas as pd

In [10]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/11.6 MB 9.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/11.6 MB 9.5 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/11.6 MB 11.1 MB/s eta 0:00:01
   ------ --------------------------------- 1.9/11.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/11.6 MB 11.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.9/11.6 MB 10.8 MB/s eta 0:00:01
   ----------- ---------------------------- 3.4/11.6 MB 10.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.6 MB 10.8 MB/s eta 0:00:01
   ------------- -------------------------- 4.0/11.6 MB 9.6 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 9.3 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.6 MB 8.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.6/11.6 MB 8.3 MB/s eta 0:00:01


In [3]:
import os
import shutil

import cv2
import numpy as np
from pycocotools.coco import COCO
from PIL import Image
import matplotlib.pyplot as plt

In [4]:

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

C:\Users\sridh\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.
C:\Users\sridh\anaconda3\envs\pytorch_env\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-8 Python-3.9.19 torch-2.4.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [8]:
coco_images_path = r'C:\Users\sridh\Downloads\val2017\val2017'
coco_annotations_path = r"C:\Users\sridh\Downloads\annotations_trainval2017\annotations\instances_val2017.json"
classes_dir = 'classes'


In [9]:
import json

with open(r'C:\Users\sridh\Downloads\annotations_trainval2017\annotations\instances_val2017.json', 'r') as f:
    data = json.load(f)


In [143]:
# Extract essential components
images = [
    {
        "id": image["id"],
        "file_name": image["file_name"],
        "height": image["height"],
        "width": image["width"]
    }
    for image in data.get('images', [])
]

annotations = [
    {
        "id": annotation["id"],
        "image_id": annotation["image_id"],
        "category_id": annotation["category_id"],
        "bbox": annotation["bbox"],
        "area": annotation["area"],
        "iscrowd": annotation["iscrowd"]
    }
    for annotation in data.get('annotations', [])
]

categories = [
    {
        "category_id": category["id"],
        "name": category["name"]
    }
    for category in data.get('categories', [])
]

clean_data = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}


with open('clean_annotations.json', 'w') as f:
    json.dump(clean_data, f, indent=4)


In [144]:
clean_data['images'][:5]

[{'id': 397133, 'file_name': '000000397133.jpg', 'height': 427, 'width': 640},
 {'id': 37777, 'file_name': '000000037777.jpg', 'height': 230, 'width': 352},
 {'id': 252219, 'file_name': '000000252219.jpg', 'height': 428, 'width': 640},
 {'id': 87038, 'file_name': '000000087038.jpg', 'height': 480, 'width': 640},
 {'id': 174482, 'file_name': '000000174482.jpg', 'height': 388, 'width': 640}]

In [145]:

images_df = pd.DataFrame(clean_data['images'])
annotations_df = pd.DataFrame(clean_data['annotations'])
categories_df = pd.DataFrame(clean_data['categories'])


In [146]:
annotations_df.head()

,id,image_id,category_id,bbox,area,iscrowd
0,1768,289343,18,"[473.07, 395.93, 38.65, 28.67]",702.10575,0
1,1773,61471,18,"[272.1, 200.23, 151.97, 279.77]",27718.47630,0
2,2551,472375,18,"[124.71, 196.18, 372.85, 356.81]",78969.31690,0
3,3186,520301,18,"[112.71, 154.82, 367.29, 479.35]",108316.66515,0
4,3419,579321,18,"[200.61, 89.65, 400.22, 251.02]",75864.53530,0


In [147]:
images_df.head()

,id,file_name,height,width
0,397133,000000397133.jpg,427,640
1,37777,000000037777.jpg,230,352
2,252219,000000252219.jpg,428,640
3,87038,000000087038.jpg,480,640
4,174482,000000174482.jpg,388,640


In [148]:
categories_df.value_counts()

category_id  name        
1            person          1
2            bicycle         1
64           potted plant    1
63           couch           1
62           chair           1
                            ..
28           umbrella        1
27           backpack        1
25           giraffe         1
24           zebra           1
90           toothbrush      1
Name: count, Length: 80, dtype: int64

In [149]:
print(annotations_df.columns)
print(images_df.columns)
print(categories_df.columns)

Index(['id', 'image_id', 'category_id', 'bbox', 'area', 'iscrowd'], dtype='object')
Index(['id', 'file_name', 'height', 'width'], dtype='object')
Index(['category_id', 'name'], dtype='object')


In [150]:
# Convert bbox column to float type
annotations_df['bbox'] = annotations_df['bbox'].apply(lambda bbox: [float(coord) for coord in bbox])


In [153]:
annotations_df= pd.merge(annotations_df,categories_df,on='category_id')

In [155]:
annotations_df.head()

,id,image_id,category_id,bbox,area,iscrowd,name
0,1768,289343,18,"[473.07, 395.93, 38.65, 28.67]",702.10575,0,dog
1,1773,61471,18,"[272.1, 200.23, 151.97, 279.77]",27718.47630,0,dog
2,2551,472375,18,"[124.71, 196.18, 372.85, 356.81]",78969.31690,0,dog
3,3186,520301,18,"[112.71, 154.82, 367.29, 479.35]",108316.66515,0,dog
4,3419,579321,18,"[200.61, 89.65, 400.22, 251.02]",75864.53530,0,dog


In [122]:
annotations_df[annotations_df['image_id'] == 25139]

,id,image_id,category_id,bbox,area,iscrowd,name
17545,588593,25139,24,"[0.0, 2.29, 341.63, 326.37]",102506.6420,0,NaN
17609,590928,25139,24,"[313.73, 28.52, 144.86, 219.92]",20949.0227,0,NaN


In [48]:
print(annotations_df['bbox'].head())


0      [473.07, 395.93, 38.65, 28.67]
1     [272.1, 200.23, 151.97, 279.77]
2    [124.71, 196.18, 372.85, 356.81]
3    [112.71, 154.82, 367.29, 479.35]
4     [200.61, 89.65, 400.22, 251.02]
Name: bbox, dtype: object


In [49]:
selected_classes=[
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]


In [50]:
for cls in selected_classes:
    os.makedirs(os.path.join(classes_dir, cls, 'fp'), exist_ok=True)
    os.makedirs(os.path.join(classes_dir, cls, 'fn'), exist_ok=True)



In [51]:
coco = COCO(coco_annotations_path)

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


In [183]:
def coco_to_xyxy(box, img_width, img_height):
    x, y, width, height = box
    x_min = x
    y_min = y
    x_max = x + width
    y_max = y + height
    
    # Normalize coordinates
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return [x_center, y_center, width, height]

In [53]:
def calculate_iou(box1, box2):
    """Calculates Intersection over Union (IoU) between two bounding boxes."""
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    xi1 = max(x1, x3)
    yi1 = max(y1, y3)
    xi2 = min(x2, x4)
    yi2 = min(y2, y4)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)

    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area if union_area > 0 else 0

    return iou

In [54]:
confidence_threshold = 0.4
iou_threshold = 0.5

In [57]:

def combine_images_side_by_side(img1, img2):
    """Combines two images side by side."""
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    if h1 > h2:
        img2 = cv2.resize(img2, (w2, h1))
    elif h2 > h1:
        img1 = cv2.resize(img1, (w1, h2))
    combined_img = cv2.hconcat([img1, img2])
    return combined_img


In [98]:
def draw_annotations(img, boxes, colors, labels):
    """
    Draws bounding boxes and labels on the image using OpenCV.

    Parameters:
    - img: Image on which to draw
    - boxes: List of bounding boxes in [x1, y1, x2, y2] format
    - colors: List of colors for the boxes
    - labels: List of labels for the boxes
    """
    for box, color, label in zip(boxes, colors, labels):
   
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)



In [184]:
def visualize_and_save_predictions(coco, model, image_path, selected_classes, image_id, annotations, confidence_threshold=0.4, iou_threshold=0.5, save_dir='results'):
    """
    Visualizes ground truth and predictions for an image using OpenCV, checks for false positives and negatives,
    and saves images to corresponding directories based on the relevant classes for the given image ID.

    Parameters:
    - coco: COCO API instance
    - model: YOLOv5 model instance
    - image_path: Path to the image file
    - selected_classes: List of relevant class names
    - image_id: ID of the image to visualize
    - annotations: List of ground truth annotations (in format: {'bbox': [x, y, width, height], 'category_id': int})
    - confidence_threshold: Minimum confidence for predictions
    - iou_threshold: IoU threshold for matching
    - save_dir: Directory to save FP and FN images
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to read image: {image_path}")
        return

    gt_boxes = []
    for annotation in annotations:
        category_id = annotation['category_id']
        category_name = coco.loadCats(category_id)[0]['name']
        if category_name in selected_classes:
            bbox = annotation['bbox']
            gt_box = [int(bbox[0]), int(bbox[1]), int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])]
            gt_boxes.append((category_name, gt_box))

    results = model(image_path)
    predictions = results.xyxy[0].cpu().numpy()

    pred_boxes = []
    for pred in predictions:
        pred_cls = model.names[int(pred[5])]
        pred_confidence = pred[4]
        pred_box = [int(pred[i]) for i in range(4)]
        if pred_confidence > confidence_threshold and pred_cls in selected_classes:
            pred_boxes.append((pred_cls, pred_box))

    gt_img = img.copy()
    pred_img = img.copy()

    draw_annotations(gt_img, [box for _, box in gt_boxes], [(0, 255, 0)] * len(gt_boxes), ['GT'] * len(gt_boxes))
    draw_annotations(pred_img, [box for _, box in pred_boxes], [(0, 0, 255)] * len(pred_boxes), ['Pred'] * len(pred_boxes))

    combined_img = combine_images_side_by_side(gt_img, pred_img)

    cv2.imshow("Ground Truth vs Predictions", combined_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    for cls in selected_classes:
        os.makedirs(os.path.join(save_dir, cls, 'fn'), exist_ok=True)
        os.makedirs(os.path.join(save_dir, cls, 'fp'), exist_ok=True)

    fn_saved = False
    fp_saved = False

    for gt_cls, gt_box in gt_boxes:
        matched = False
        for pred_cls, pred_box in pred_boxes:
            if gt_cls == pred_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
            save_path_fn = os.path.join(save_dir, gt_cls, 'fn', os.path.basename(image_path))
            cv2.imwrite(save_path_fn, combined_img)
            print(f"Saved False Negative to: {save_path_fn}")
            fn_saved = True

    for pred_cls, pred_box in pred_boxes:
        matched = False
        for gt_cls, gt_box in gt_boxes:
            if pred_cls == gt_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
            save_path_fp = os.path.join(save_dir, pred_cls, 'fp', os.path.basename(image_path))
            cv2.imwrite(save_path_fp, combined_img)
            print(f"Saved False Positive to: {save_path_fp}")
            fp_saved = True

    if not fn_saved:
        print("No False Negatives were saved.")
    if not fp_saved:
        print("No False Positives were saved.")


In [185]:
image_path1 = r"E:\yolo\val2017\val2017\000000002299.jpg"
image_id = 2299
class_name = 'person'
selected_classes = [class_name]
annotations_filtered = annotations_df[(annotations_df['image_id'] == image_id) & (annotations_df['name'] == class_name)]
annotations_list = annotations_filtered.apply(lambda row: {'bbox': row['bbox'], 'category_id': row['category_id']}, axis=1).tolist()
visualize_and_save_predictions(coco, model, image_path1, selected_classes, image_id, annotations_list)


C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Negative to: results\person\fn\000000002299.jpg
Saved False Positive to: results\person\fp\000000002299.jpg
Saved False Positive to: results\person\fp\000000002299.jpg
Saved False Positive to: results\person\fp\000000002299.jpg


# Iterate to every coco image in the dataset and check the corrections and save to their respective folders

In [70]:
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_path, img_info['file_name'])

    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
        continue

    img_width, img_height = img.shape[1], img.shape[0]

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    gt_boxes = []
    for ann in anns:
        cls_name = coco.loadCats(ann['category_id'])[0]['name']
        if cls_name in selected_classes:
            gt_boxes.append((cls_name, coco_to_xyxy(ann['bbox'])))

    results = model(img_path)
    predictions = results.xyxy[0].numpy()

    pred_boxes = []
    for pred in predictions:
        pred_cls = model.names[int(pred[5])]
        pred_confidence = pred[4]
        pred_box = [int(pred[i]) for i in range(4)]
        if pred_confidence > confidence_threshold:
            pred_boxes.append((pred_cls, pred_box))

    gt_img = img.copy()
    draw_annotations(gt_img, [box for _, box in gt_boxes], [(0, 255, 0)] * len(gt_boxes), ['GT'] * len(gt_boxes))

    pred_img = img.copy()
    draw_annotations(pred_img, [box for _, box in pred_boxes], [(0, 0, 255)] * len(pred_boxes), ['Pred'] * len(pred_boxes))


    combined_img = combine_images_side_by_side(gt_img, pred_img)


    for pred_cls, pred_box in pred_boxes:
        matched = False
        for gt_cls, gt_box in gt_boxes:
            if pred_cls == gt_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
     
            save_path_fp = os.path.join(classes_dir, pred_cls, 'fp', os.path.basename(img_path))
            cv2.imwrite(save_path_fp, combined_img)
            break  

    for gt_cls, gt_box in gt_boxes:
        matched = False
        for pred_cls, pred_box in pred_boxes:
            if gt_cls == pred_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
            # Save as false negative
            save_path_fn = os.path.join(classes_dir, gt_cls, 'fn', os.path.basename(img_path))
            cv2.imwrite(save_path_fn, combined_img)
            break  # Move to next image after saving false negative

print("Process completed.")

C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models

Process completed.
